# [12 Python Decorators to Take Your Code to the Next Level](https://towardsdatascience.com/12-python-decorators-to-take-your-code-to-the-next-level-a910a1ab3e99)

## 1 — @logger (to get started)✏️

Если вы впервые знакомы с декораторами, то можете считать их функциями, которые принимают функции на вход и расширяют их функциональные возможности без изменения их основного назначения.

Давайте начнем с простого декоратора, который расширяет функцию, регистрируя время начала и окончания ее выполнения.

Результат работы декорируемой функции будет выглядеть следующим образом:

# ----- some_function: start -----
# some_function executing
# ----- some_function: end -----
Чтобы написать этот дешифратор, сначала нужно выбрать подходящее имя: назовем его logger.

Логгер - это функция, которая принимает функцию на вход и возвращает функцию на выход. Выходная функция обычно является расширенной версией входной. В нашем случае мы хотим, чтобы выходная функция окружала вызов входной функции утверждениями start и end.

Поскольку мы не знаем, какие аргументы использует входная функция, мы можем передать их из функции-обертки с помощью выражений *args и **kwargs. Эти выражения позволяют передавать произвольное количество позиционных и ключевых аргументов.

Вот простая реализация декоратора регистратора:

In [1]:

def logger(function):
    def wrapper(*args, **kwargs):
        print(f"----- {function.__name__}: start -----")
        output = function(*args, **kwargs)
        print(f"----- {function.__name__}: end -----")
        return output
    return wrapper

@logger
def some_function(text):
    print(text)

some_function("first test")
# ----- some_function: start -----
# first test
# ----- some_function: end -----
some_function("second test")
# ----- some_function: start -----
# second test
# ----- some_function: end -----


----- some_function: start -----
first test
----- some_function: end -----
----- some_function: start -----
second test
----- some_function: end -----


## 2 — @wraps 🎁
Этот декоратор обновляет функцию-обертку, чтобы она выглядела как оригинальная функция и наследовала ее имя и свойства.

Чтобы понять, что делает @wraps и почему вы должны его использовать, давайте возьмем предыдущий декоратор и применим его к простой функции, которая складывает два числа.

In [2]:
def logger(function):
    def wrapper(*args, **kwargs):
        """wrapper documentation"""
        print(f"----- {function.__name__}: start -----")
        output = function(*args, **kwargs)
        print(f"----- {function.__name__}: end -----")
        return output
    return wrapper

@logger
def add_two_numbers(a, b):
    """this function adds two numbers"""
    return a + b

In [5]:
print(add_two_numbers.__name__)

print(add_two_numbers.__doc__)

wrapper
wrapper documentation


Если мы проверим имя и документацию декорированной функции add_two_numbers, вызвав атрибуты `__name__` и `__doc__`, мы получим ... неестественные (и все же ожидаемые) результаты:

Вместо этого мы получаем имя обертки и документацию ⚠️.

Это нежелательный результат. Мы хотим сохранить имя и документацию исходной функции. Вот тут-то и пригодится декоратор @wraps.

Все, что вам нужно сделать, это украсить функцию-обертку.

In [6]:
from functools import wraps

def logger(function):
    @wraps(function)
    def wrapper(*args, **kwargs):
        """wrapper documentation"""
        print(f"----- {function.__name__}: start -----")
        output = function(*args, **kwargs)
        print(f"----- {function.__name__}: end -----")
        return output
    return wrapper

@logger
def add_two_numbers(a, b):
    """this function adds two numbers"""
    return a + b

In [7]:
print(add_two_numbers.__name__)

print(add_two_numbers.__doc__)

add_two_numbers
this function adds two numbers


## 3 — @lru_cache 💨

Это встроенный декоратор, который можно импортировать из functools.

Он кэширует возвращаемые значения функции, используя алгоритм наименее часто используемых значений (LRU) для отбрасывания наименее используемых значений, когда кэш заполняется.

Обычно я использую этот декоратор для длительно выполняющихся задач, которые не меняют результат при одинаковых входных данных, например, запрос к базе данных, запрос статической удаленной веб-страницы или тяжелая обработка.

В следующем примере я использую lru_cache для декорирования функции, которая имитирует некоторую обработку. Затем я применяю функцию на одном и том же входе несколько раз подряд.

In [9]:
import random
import time
from functools import lru_cache


@lru_cache(maxsize=None)
def heavy_processing(n):
    sleep_time = n + random.random()
    time.sleep(sleep_time)

# first time
%%time
heavy_processing(0)
# CPU times: user 363 µs, sys: 727 µs, total: 1.09 ms
# Wall time: 694 ms

# second time
%%time
heavy_processing(0)
# CPU times: user 4 µs, sys: 0 ns, total: 4 µs
# Wall time: 8.11 µs

# third time
%%time
heavy_processing(0)
# CPU times: user 5 µs, sys: 1 µs, total: 6 µs
# Wall time: 7.15 µs

UsageError: Line magic function `%%time` not found.


In [15]:
%%time
heavy_processing(0)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 4.77 µs
